In [7]:
#데이터수집단계

! pip install opencv-python
import cv2
 
# open webcam (웹캠 열기)
webcam = cv2.VideoCapture(0)
 
if not webcam.isOpened():
    print("Could not open webcam")
    exit()
    
#mode를 nomask로 설정했을 시, 마스크 안 쓴 얼굴이미지 데이터 수집(웹캠으로 수집)
#mode를 mask로 설정했을 시, 마스크 쓴 얼굴이미지 데이터 수집(웹캠으로 수집)
#데이터셋 직접 촬영

mode = "mask"

sample_num = 0    
captured_num = 0

#mode마다 각각 찍을 사진 장수
num = 200
    
# loop through frames
while webcam.isOpened():
    
    # read frame from webcam 
    status, frame = webcam.read()
    sample_num = sample_num + 1
    
    if not status:
        break
 
    # display output
    cv2.imshow("captured frames", frame)
    
    #mask mode일 경우, mask_images 폴더에 이미지 데이터 수집
    if mode == "mask" and sample_num == 4:
        captured_num = captured_num + 1
        cv2.imwrite('C:/Users/dufma/mask_images/img'+str(captured_num)+'.jpg', frame) #마스크 쓴 얼굴이미지 저장
        sample_num = 0
        
    #nomask mode일 경우, images폴더에 이미지 데이터 수집    
    elif mode == "nomask" and sample_num == 4:
        captured_num = captured_num + 1
        cv2.imwrite('C:/Users/dufma/nomask_images/img'+str(captured_num)+'.jpg', frame) #마스크 안 쓴 얼굴이미지 저장
        sample_num = 0
        
        
    
    # press "Q" 키 눌러서 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    # 앞에서 지정한 장수만큼 촬영 후 웹캠 자동종료
    if captured_num == num:
        break
# release resources
webcam.release()
cv2.destroyAllWindows()   

In [8]:
#데이터 전처리 및 모델링 학습

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

 
#마스크 착용/미착용 데이터셋 폴더 경로 설정    
path_dir1 = 'C:/Users/dufma/nomask_images/'
path_dir2 = 'C:/Users/dufma/mask_images/'
 
file_list1 = os.listdir(path_dir1) # path에 존재하는 파일 목록 가져오기
file_list2 = os.listdir(path_dir2)
 
file_list1_num = len(file_list1)
file_list2_num = len(file_list2)
 
file_num = file_list1_num + file_list2_num
 
#%% 이미지 전처리
num = 0;
all_img = np.float32(np.zeros((file_num, 224, 224, 3))) 
all_label = np.float64(np.zeros((file_num, 1)))
 
for img_name in file_list1:
    img_path = path_dir1+img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 0 # nomask
    num = num + 1
    
for img_name in file_list2:
    img_path = path_dir2+img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 1 # mask
    num = num + 1
 
 
# 데이터셋 랜덤으로 섞기(적절하게 훈련되게 하기 위함) 
n_elem = all_label.shape[0]
indices = np.random.choice(n_elem, size=n_elem, replace=False)
 
all_label = all_label[indices]
all_img = all_img[indices]
 
    
# 훈련셋 테스트셋 분할
num_train = int(np.round(all_label.shape[0]*0.8))
num_test = int(np.round(all_label.shape[0]*0.2))
 
train_img = all_img[0:num_train, :, :, :]
test_img = all_img[num_train:, :, :, :] 
 
train_label = all_label[0:num_train]
test_label = all_label[num_train:]
 
 
 
#%% 
# create the base pre-trained model
IMG_SHAPE = (224, 224, 3)

#base model 을 ResNet50모델로 설정 후 학습
base_model = ResNet50(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
base_model.trainable = False
base_model.summary()
print("Number of layers in the base model: ", len(base_model.layers))
 
flatten_layer = Flatten()
#활성화 함수 relu 사용
dense_layer1 = Dense(128, activation='relu')
bn_layer1 = BatchNormalization()
dense_layer2 = Dense(1, activation=tf.nn.sigmoid)
 
model = Sequential([
        base_model,
        flatten_layer,
        dense_layer1,
        bn_layer1,
        dense_layer2,
        ])
 
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()
 
model.fit(train_img, train_label, epochs=20, batch_size=16, validation_data = (test_img, test_label))

# save model
model.save("model.h5")
 
print("Saved model to disk")  

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               12845184  
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 36,433,537
Trainable params: 12,845,569
Non-trainable params: 23,587,968
_________________________________________________________________
Epoch 1/20
20/20 [==============================] - 54s 2s/step - loss: 0.0544 - accuracy: 0.9719 -

In [6]:
!pip install cvlib

  Using cached cvlib-0.2.6.tar.gz (10.0 MB)
  Using cached progressbar-2.5.tar.gz (10 kB)
  Using cached imutils-0.5.4.tar.gz (17 kB)
  Created wheel for cvlib: filename=cvlib-0.2.6-py3-none-any.whl size=10044628 sha256=d1973de4d23a2d57567db8464386ec03fa1060579f3cdcd7ff0cc70024d97116
  Stored in directory: c:\users\dufma\appdata\local\pip\cache\wheels\31\dc\ae\94e2060a8bed427176312a72061d55a9cc65f5a1883934505e
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25862 sha256=41e9e98f4716d3aac7d4f746666f0e2c7535b84ff3f29d2f884e137935357092
  Stored in directory: c:\users\dufma\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12078 sha256=1d15ff0be202688e3d5ab439cfa7772fa5cb02afd74bc1b992d780878b9236c5
  Stored in directory: c:\users\dufma\appdata\local\pip\cache\wheels\d7\d9\89\a3f31c76ff6d51dc3b1575628f59afe59e4ceae3f2748cd7ad
Successfully built

In [9]:
#웹캠으로 mask detection

import cvlib as cv
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import ImageFont, ImageDraw, Image
 
    
#저장한 model.h5 불러오기    
model = load_model('model.h5')
model.summary()
 
# open webcam
webcam = cv2.VideoCapture(0)
 
    
if not webcam.isOpened():
    print("Could not open webcam")
    exit()
    
    
# loop through frames
while webcam.isOpened():
 
    # read frame from webcam 
    status, frame = webcam.read()
    
    if not status:
        print("Could not read frame")
        exit()
 
    # apply face detection
    face, confidence = cv.detect_face(frame)
 
    # loop through detected faces
    # 여러명 동시에 detect 하기위해 loop문 사용
    for idx, f in enumerate(face):
        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
        
        if 0 <= startX <= frame.shape[1] and 0 <= endX <= frame.shape[1] and 0 <= startY <= frame.shape[0] and 0 <= endY <= frame.shape[0]:
            
            face_region = frame[startY:endY, startX:endX]
            
            face_region1 = cv2.resize(face_region, (224, 224), interpolation = cv2.INTER_AREA)
            
            x = img_to_array(face_region1)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            
            prediction = model.predict(x)
 
            if prediction < 0.5: # 마스크 착용으로 판별되면, 
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,0,255), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "No Mask ({:.2f}%)".format((1 - prediction[0][0])*100)
                cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                
            else: # 마스크 착용으로 판별되면
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "Mask ({:.2f}%)".format(prediction[0][0]*100)
                cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                
    # display output(웹캠으로 마스크 유무 식별)
    cv2.imshow("mask nomask classify", frame)
 
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# release resources
webcam.release()
cv2.destroyAllWindows() 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               12845184  
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 36,433,537
Trainable params: 12,845,569
Non-trainable params: 23,587,968
_________________________________________________________________
